# Time-Step: hourly, bi-hourly, tri-hourly, etc.
This notebook presents an example of the openTEPES' features: Time-Step. This feature controls the time domain's resolution, either hourly or bi-hourly, etc.
The main point is that if Time-Step is not equal to 1, openTEPES compute the demand as the mean over the time step load levels and assign it to active load levels. Idem for operating reserve, variable max power, variable min, and max storage and inflows. For example, we perfom the next comparison.

## Changing TimeStep equal to 1

In [1]:
import altair as alt
import pandas as pd
from openTEPES.openTEPES import openTEPES_run

### Defining the DirName, CaseName, Solvername, and IndLogConsole.

In [2]:
DirName =''
CaseName = '9n'
SolverName = 'gurobi'
IndLogConsole = 'No'

In the folder called ``9n``, you can find the CSV file ``oT_Data_Parameter_9n`` which is also shown as follows:

In [3]:
df = pd.read_csv(CaseName+'/oT_Data_Parameter_'+CaseName+'.csv', index_col=[0])
df.head()

,ENSCost,CO2Cost,UpReserveActivation,DwReserveActivation,MinRatioDwUp,MaxRatioDwUp,SBase,ReferenceNode,TimeStep
Parameters,10000,25,0.25,0.3,0,1,100,Node_4,1


This CSV file comprises all the system parameters of the openTEPES mode. It has a column called TimeStep, that having ``1`` for hourly resolution, ``2`` for bi-hourly resolution, etc. In our example, we consider a TimeStep equal to ``3`` (tri-hourly resolution) and equal to ``4`` (tetra-hourly resolution).

### Having hourly resolution

In [4]:
df['TimeStep'] = 1

In [5]:
df.to_csv(CaseName+'/oT_Data_Parameter_'+CaseName+'.csv', index=True)

In [6]:
model_TS1 = openTEPES_run(DirName, CaseName, SolverName, IndLogConsole)

Input data                             ****


C:\Users\erik_\miniconda3\lib\site-packages\pandas\core\frame.py:5176: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(
C:\Users\erik_\miniconda3\lib\site-packages\pyomo\core\base\block.py:541: PerformanceWarning: indexing past lexsort depth may impact performance.
  self.add_component(name, val)


Reading    input data                  ...  0 s
Setting up input data                  ...  20 s
Setting up variables                   ...  8 s
Investment           model formulation ****
Generation operation model formulation ****
Network    switching model formulation ****
Network    operation model formulation ****
Writing LP file                        ...  60 s
Problem solving                        ****
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: x1712273
  Lower bound: 136.41411098938326
  Upper bound: 136.41411098938326
  Number of objectives: 1
  Number of constraints: 899782
  Number of variables: 1108851
  Number of binary variables: 0
  Number of integer variables: 0
  Number

### Getting the total output per technology

In [7]:
OutputToFile_TS1 = pd.Series(data=[sum(model_TS1.vTotalOutput[sc,p,n,g]() for g in model_TS1.g if (gt,g) in model_TS1.t2g)*model_TS1.pTimeStep() for sc,p,n,gt in model_TS1.sc*model_TS1.p*model_TS1.n*model_TS1.gt], index=pd.MultiIndex.from_tuples(model_TS1.sc*model_TS1.p*model_TS1.n*model_TS1.gt))
OutputToFile_TS1 = OutputToFile_TS1.to_frame(name='GW' ).reset_index().pivot_table(index=['level_0','level_1','level_2'], columns='level_3', values='GW').rename_axis(['Scenario','Period','LoadLevel'], axis=0).rename_axis([None], axis=1).sum(axis=0)

In [8]:
OutputToFile_TS1.index.names = ['Technology']
OutputToFile_TS1 = OutputToFile_TS1.to_frame(name='GWh')
OutputToFile_TS1['Resolution'] = 'TimeStep-1'
OutputToFile_TS1 = OutputToFile_TS1.reset_index()

In [9]:
OutputToFile_TS1['GWh'] = round(OutputToFile_TS1['GWh'], 2)

In [10]:
chart = alt.Chart(OutputToFile_TS1).mark_bar().encode(x='Technology:N', y='GWh:Q', color='Technology:N', column='Resolution:O')
# chart.save(CaseName+'/oT_Plot_TechnologyOutputComparison_'+CaseName+'.html', embed_options={'renderer':'svg'})

In [11]:
chart

alt.Chart(...)

## Comparison of the total output per technology considering TimeStep equal to 3 and 4
An additional example can be found in the following notebook:

- [2.1-Comparison.ipynb](2.1-Comparison.ipynb)